## Logic Programming

Logic programming is a programming paradigm based on logic. A program herein is a set of logical expressions stating facts and rules.

A **fact** is some an atomic formula (with no logic operations, except negation) that is assumed true.

A **rule** is an implication with form

$$H \leftarrow B_1, B_2, \ldots, B_n$$

where $B_i$ are atomic formulas that need to be true in order to $H$ to be true (sometimes $H$ is called the head and the set of $B_i$ the body).

This rule can be seen as

+ a logical implication: $B_1$ and $B_2$ and ... implies $H$

+ a program: to solve $H$ you must solve $B_1$, then $B_2$, ...

To solve a certain problem we need to write it into a collection of rules and facts, that is, we need to translate it into a logical statement.

This statement is what is called a **logic program**. A computation over a logic program is the deduction of some of its implications/consequences.

**Querying** a logic program is finding if some logic expression is one of the program's consequences. Usually, in answering the query we also get a model, that is, the concrete values that makes the expression a consequence of the program.

These queries are computed via a backtracking algoritm called **unification** that attempts to match parts of the query with clauses from the available rules and facts. If the algorithm is able to match the query it succeeds, otherwise it fails.

Due to the backtracking nature of unification, the algorithm is able to provide all possible combinations of facts that satisfy a given query.

Despite the declarative nature of logic programs, there is also an imperative perspective. Unification, while replacing (or _rewriting_) part of the query with a possible match, can be seen as performing a tree depth-first search over the collection of rules and facts. So, the order of how the rules (and the clauses within rules) are listed on the program determines how the tree search will occur. 

Different orderings might result in enormous differences in efficiency, and even in program correction due to nontermination (if depth-first becomes locked inside the exploration of an infinite tree branch). This is a serious [leaky abstraction](https://en.wikipedia.org/wiki/Leaky_abstraction) that logic programmers must always consider.

Logic languages also have an operator that allows the search termination on some branches of the tree (because the programmer knows that a search there will be useless). Also, the judicious use of this operator (called the cut operator), used for computation eficiency and/or termination, represents a side-effect that contaminates the pure declarative nature of a logic program.



---



The most famous example of a Logic Programming language is Prolog. 

Here we will use the Prolog implemetation, `swipl`, which has an interface to Python, `pyswip`:

In [ ]:
# On Windows:
# 1. install swipl @ https://www.swi-prolog.org/
# 2. add swipl exe to PATH
!sudo apt install swi-prolog # install swipl on Colab

In [ ]:
!pip install pyswip

An object of class `Prolog` gives us a Prolog engine to query the provided rules and facts.

In [ ]:
from pyswip import Prolog

# subclass to replace the standard: program = Prolog()
# because the clause cache does not reset, so we 
# need to do stuff like this, everytime we start
  # program.retractall('father(_,_)');
  # program.retractall('grandfather(_,_)');
class LP(Prolog):
  def __init__(self, terms=[]):
    super().__init__()
    for term in terms:
      name, arity = term.split('/')
      clause = name + '(' + ','.join('_'*int(arity)) + ')'
      self.retractall(clause)
  
  # check if query is satisfiable
  def isSat(self, aQuery):
    return list(program.query(aQuery)) != []

ImportError: ignored

Logic programming consists on the use of three types of statements

+ **facts**: a statement that some relation exists, like _alex is the father of michael and michael is the father of gina_

+ **rules**: a statement that creates a new abstract relation between other facts and rules, like _if X is father of Y, and Y is father of Z, then X is grandfather of Z_. 

+ **queries**: a statement that asks if some expression is a consequence of the program, like _is alex the gradfather of gina?_ or _is there someone that is father of gina?_ It's usual for the engine to return all models that make the query a consequence of the program

These statements are build from **terms** like

+ **constants**, fixed values, like number or strings

+ **logical variables**, used by the engine to assign values in order to satisfy queries. By convention, logical variables start in uppercase letters.

+ **functors**, which denote relations. Functors have structure `functor-name(term1, term2, ..., termn)`. The number of parameters is the functor's **arity**. Constants are just functors with zero arity



Let's see an example of a logical program with these three types of statements,

In [ ]:
               # list of program's functors
program = LP( ['father/2', 'grandfather/2'] ) 

program.assertz("father(alex,michael)")                         # facts
program.assertz("father(michael,john)")
program.assertz("father(michael,gina)")

program.assertz("grandfather(X,Y) :- father(X,Z), father(Z,Y)") # rule

for soln in program.query("father(X,Y)"):                       # queries
    print(soln["X"], "is the father of", soln["Y"]) # soln is the current model

print()
for soln in program.query("grandfather(X,Y)"):
    print(soln["X"], "is the grandfather of", soln["Y"])

print()
for soln in program.query("father(X,gina)"):
    print(soln["X"], "is the father of gina")    

NameError: ignored

Notice that a rule like

$$\text{grandfather}(X,Y) \leftarrow \text{father}(X,Z), \text{father}(Z,X)$$

All variables are universally quantified variables, but $Z$ can be seen as existencially quantified. The rule can be read as _for all X,Y; X is the grandfather of Y if there exists a Z such that X is father of Z and Z is father of Y_ 

If the query is not satisfiable, the engine will provide an empty list of models

In [ ]:
print( program.isSat('father(X,gina)')    )
print( program.isSat('father(alex,gina)') )

True
False


Let's add a rule for siblings,

In [ ]:
program = LP( ['father/2', 'sibling/2'] ) 

program.assertz("father(alex,michael)")
program.assertz("father(michael,john)")
program.assertz("father(michael,gina)")

program.assertz("sibling(X,Y) :- father(Z,X), father(Z,Y)")

for soln in program.query("sibling(X,Y)"):
    print(soln["X"], "is a sibling of", soln["Y"])

michael is a sibling of michael
john is a sibling of john
john is a sibling of gina
gina is a sibling of john
gina is a sibling of gina


There are some wrong answers, because the engine is matching a person as her own sibling, since everybody trivially shares his father with herself.

The meaning of the previous logic program was not what we wanted,

In [ ]:
program = LP( ['father/2', 'sibling/2'] ) 

program.assertz("father(alex,michael)")
program.assertz("father(michael,john)")
program.assertz("father(michael,gina)")

program.assertz("sibling(X,Y) :- father(Z,X), father(Z,Y), X \== Y")

for soln in program.query("sibling(X,Y)"):
    print(soln["X"], "is a sibling of", soln["Y"])

john is a sibling of gina
gina is a sibling of john


Terms can be recursive,

In [ ]:
program = LP( ['parent/2', 'ancestor/2'] ) 

program.assertz("parent(alex,michael)")
program.assertz("parent(michael,john)")
program.assertz("parent(michael,gina)")
program.assertz("parent(gina,bob)")

program.assertz("ancestor(X,Y) :- parent(X,Y)")
program.assertz("ancestor(X,Y) :- father(X,Z), ancestor(Z,Y)")

for soln in program.query("ancestor(X,Y)"):
    print(soln["X"], "is a ancestor of", soln["Y"])

alex is a ancestor of michael
michael is a ancestor of john
michael is a ancestor of gina
gina is a ancestor of bob
alex is a ancestor of john
alex is a ancestor of gina
alex is a ancestor of bob
michael is a ancestor of bob


Prolog includes lists,

In [ ]:
program = LP( ['member/2', 'double/2'] ) 

program.assertz("double(L,L2) :- append(L,L,L2)")

program.assertz("member(X,[X|Xs])")
program.assertz("member(X,[Y|Ys]) :- member(X,Ys)")

print(program.isSat("double([1,2,3],L), member(3,L)"))

True


The `member` rule is a typical predicate for list membership. 

But we can use this rule in a different way,

In [ ]:
for soln in program.query("member(X,[1,2,3])"):
    print(soln["X"], "is a list member")

1 is a list member
2 is a list member
3 is a list member


Or even asking how to replace a variable inside the list to satisfy the query,

In [ ]:
for soln in program.query("member(3,[1,3,Z])"):
    print(soln["Z"], "is a list member")

_2174 is a list member
3 is a list member


We can make it radical and ask what are the lists where 3 is a member? Well, there are infinite answers, so let's just compute the first ones,

In [ ]:
iter = program.query("member(3,L)")
for _ in range(6):
  soln = next(iter)
  print(soln["L"], "is a possible list")

[3] is a possible list
[Variable(101), 3] is a possible list
[Variable(101), Variable(102), 3] is a possible list
[Variable(101), Variable(102), Variable(103), 3] is a possible list
[Variable(101), Variable(102), Variable(103), Variable(104), 3] is a possible list
[Variable(101), Variable(102), Variable(103), Variable(104), Variable(105), 3] is a possible list


The first model states that $Z$ can be any value (here described by a new variable created by the engine) because there's already a 3 in the list. The second model states that $Z=3$ also satisfies the query.

This variable creation can be seen in these simple queries,

In [ ]:
program = LP() 

for soln in program.query("X = Y"):
    print("X=", soln["X"], " Y=", soln["Y"])

for soln in program.query("X \== Y"):
    print("X=", soln["X"], " Y=", soln["Y"])    

X= _1750  Y= _1750
X= _1752  Y= _1754


A rule with no conditions is universally satisfiable,

In [ ]:
program = LP( ['thing/1'] ) 

program.assertz("thing(X)")  # anything is a thing

program.isSat("thing(prolog)")

True

+ Exercise: create a rule that produces the permutations of a given list,

In [ ]:
program = LP( ['remove/3', 'permutation/2'] ) 

program.assertz("remove(X,[X|Xs],Xs)") 
program.assertz("remove(X,[Y|Ys],[Y|Zs]) :- remove(X,Ys,Zs)") 

program.assertz("permutation([],[])") 
program.assertz("permutation(Xs,[Z|Zs]) :- remove(Z,Xs,Ys), permutation(Ys,Zs)") 

for soln in program.query("permutation([1,2,3],L)"):
  print("L=", soln["L"])   

L= [1, 2, 3]
L= [1, 3, 2]
L= [2, 1, 3]
L= [2, 3, 1]
L= [3, 1, 2]
L= [3, 2, 1]




---



### Negation as failure

Logic programs describe what is considered to be true.

However, it is natural to express rules like

$$\text{bachelor}(X) \leftarrow \text{male(X)}, \text{not} ~\text{married(X)}$$

What is the semantics of the `not` operator? The perspective to deal with negation is to consider it as a failure. A goal $G$ fails (i.e., $\neg G$  succeeds), if $G$ is not a consequence of the program.

This is not the same as the negation of first-order logic. We are not checking if $\neg G$ is a consequence of the program. We are just checking if $G$ is a consequence, and it that effort fails, we _assume_ $\neg G$ is true. Otherwise, if $G$ succeeds then the rule that depends on $\neg G$ fails.

### Arithmetic in Prolog

Prolog uses operator `is` to assign a value to a variable. Also, Prolog includes the typical list of arithmetic operators,

In [ ]:
program = LP( ['plus/3', 'gcd/3', 'factorial/2'] ) 

program.assertz("plus(X,Y,Z) :- Z is X+Y") 

# Greatest Common Divider using Euclid algorithm
program.assertz("gcd(I,0,I)") 
program.assertz("gcd(I,J,G) :- J>0, R is I mod J, gcd(J,R,G)") 

program.assertz("factorial(0,1)") 
program.assertz("factorial(N,F) :- N>0, N1 is N-1, factorial(N1,F1), F is N*F1") 

for soln in program.query("plus(12,18,S)"):
  print("S =", soln["S"])   

for soln in program.query("gcd(12,18,G)"):
  print("G =", soln["G"])   

for soln in program.query("factorial(6,F)"):
  print("F =", soln["F"])

S = 30
G = 6
F = 720


### Cuts

As mentioned earlier, Prolog will depth-first the tree of rules and facts to satisfy a given query.

However, if the tree is large, this search will have performance problems. Some of these problems can be avoided if the program is allowed to tell Prolog to stop searching some parts of the tree. That's the purpose of the cut operator (denoted as `!`).

Consider the task of merging two ordered lists. For each pair of first elements from both lists, $X, Y$, only one of the next expressions will be true: $X<Y, X=Y, X>Y$. So, if one of this is satisfied, we can safely tell Prolog to stop searching the next rules.

In [ ]:
program = LP( ['merge/3'] ) 

program.assertz("merge([X|Xs],[Y|Ys],[X|Zs]) :- X < Y, !, merge(   Xs,[Y|Ys],Zs)") 
program.assertz("merge([X|Xs],[Y|Ys],[X|Zs]) :- X=:=Y, !, merge(   Xs,   Ys ,Zs)") 
program.assertz("merge([X|Xs],[Y|Ys],[Y|Zs]) :- X > Y, !, merge([X|Xs],  Ys ,Zs)") 

program.assertz("merge(Xs,[],Xs) :- !") # prevent redundant solution of merge([],[])
program.assertz("merge([],Ys,Ys) :- !")

for soln in program.query("merge([1,3,5],[2,4,7,8],M)"):
  print("M =", soln["M"])

for soln in program.query("merge([],[],M)"):
  print("M =", soln["M"])  

M = [1, 2, 3, 4, 5, 7, 8]
M = []


A cut prunes all same-named rules below. It also prunes branches that could result on evaluating clauses to its left. But Prolog will still evaluate branches of clauses to its right. However if that rule fails after the cut, no more solutions will be searched.

With this knowledge, we could have implemented the not operator,

      not(X) :- X, !, fail
      not(X)

or a different variable operator `≠`

      ≠(X,X) :- !, fail
      ≠(X,Y)

(`fail` is a Prolog predicate that always fails)

A cut that does not alter the program's meaning is denoted _green cut_. Otherwise it's denoted _red cut_.

The cuts in the two previous pseudo-codes are red cuts.

But most red cuts are not benign... The following rule for computing the minimum is wrong

    minimum(X,Y,X) :- X <= Y, !
    minimum(X,Y,Y)

the program would succeed `mimimum(2,5,5)`. The programmer needs to make explicit the unification of the 1st and 3rd argument,

    minimum(X,Y,Z) :- X <= Y, !, Z=X
    minimum(X,Y,Y)

The use of cuts must be done with extreme care. It is very easy to introduce unwanted and yet subtle behaviors in a logic program.

### Interaction between Python and Prolog

With `pyswip` it's possible to assign atomic formulas to Python functions. The next example inserts a print side-effect:

In [ ]:
#Using foreign functions within the logical program
from pyswip import Prolog, registerForeign

def hello(t):
    print("Hello,", t)
hello.arity = 1
registerForeign(hello)

prolog = LP({'father':2, 'grandfather':2})
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
for sol in prolog.query("father(michael,X), hello(X)"):
  pass

Hello, john
Hello, gina


And here's a Python function helping printing the solution of the Hanoi's Towers puzzle,

In [ ]:
def notify(t):
    print("move disk from %s pole to %s pole." % tuple(t))
notify.arity = 1
registerForeign(notify)

hanoi = LP({'hanoi':1, 'move':4})
hanoi.assertz("hanoi(N) :- move(N, left, right, center)")
hanoi.assertz("move(0, _, _, _) :- !")
hanoi.assertz("""
move(N, A, B, C) :-
  M is N-1,
  move(M, A, C, B),
  notify([A,B]),
  move(M, C, B, A)
""")

N = 3
list( hanoi.query(f"hanoi({N})") );

move disk from left pole to right pole.
move disk from left pole to center pole.
move disk from right pole to center pole.
move disk from left pole to right pole.
move disk from center pole to left pole.
move disk from center pole to right pole.
move disk from left pole to right pole.


To run a Prolog program from a text file:

In [ ]:
program = """
hanoi(N) :- move(N, left, right, center).
move(0, _, _, _) :- !.
move(N, A, B, C) :-
  M is N-1,
  move(M, A, C, B),
  notify([A,B]),
  move(M, C, B, A).
"""

f = open('hanoi.pl', 'w')
f.write(program)
f.close()

In [ ]:
hanoi = LP({'hanoi':1, 'move':4})
registerForeign(notify)

prolog.consult("hanoi.pl")
N = 4
list( hanoi.query(f"hanoi({N})") );

move disk from left pole to center pole.
move disk from left pole to right pole.
move disk from center pole to right pole.
move disk from left pole to center pole.
move disk from right pole to left pole.
move disk from right pole to center pole.
move disk from left pole to center pole.
move disk from left pole to right pole.
move disk from center pole to right pole.
move disk from center pole to left pole.
move disk from right pole to left pole.
move disk from center pole to right pole.
move disk from left pole to center pole.
move disk from left pole to right pole.
move disk from center pole to right pole.




---



## References

+ Leon Sterling et al., The Art of Prolog 2ed (1993)

+ Patrick Blackburn et al., [SWI-Prolog tutorial](http://lpn.swi-prolog.org/lpnpage.php?pagetype=html&pageid=lpn-html)

+ Yüce Tekol, [PySwip @ GitHub](https://github.com/yuce/pyswip/tree/master/examples), Python interface to SWI-Prolog



